# Exploring Langchain

## 1. Structured Outputs


#### A. Tool calling

In [1]:
from pydantic import BaseModel, Field
class ResponseFormatter(BaseModel):
    """Always use this tool to structure your response to the user."""
    answer: str = Field(description="The answer to the user's question")
    followup_question: str = Field(description="A followup question the user could ask")


from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
model_with_tools = model.bind_tools([ResponseFormatter])

ai_msg = model_with_tools.invoke("What's the difference between fussion and fission in nuclear energy and which is easier")
ai_msg


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yy4YHYpoTR0NH83fVEBiOQhw', 'function': {'arguments': '{"answer": "Nuclear fission is the process of splitting a heavy atomic nucleus into smaller nuclei, along with the release of energy. This process is commonly used in nuclear reactors and atomic bombs. In contrast, nuclear fusion is the process of combining two light atomic nuclei to form a heavier nucleus, which also releases energy. Fusion is the process that powers the sun and other stars.\\n\\nIn terms of ease, fission is currently easier to achieve and control with existing technology, as it has been extensively developed for power generation. Fusion, while it has the potential to provide a nearly limitless source of energy, is much more challenging to achieve and maintain under controlled conditions, as it requires extremely high temperatures and pressures to overcome the repulsive forces between the positively charged nuclei.", "followup_question": "What are

In [6]:
ai_msg.tool_calls[1]["args"]

{'answer': 'Fission is currently used in nuclear power plants, while fusion is still largely experimental. Fission reactors are well-established and can generate significant amounts of electricity. Fusion, on the other hand, has not yet been successfully harnessed for practical energy production, although research is ongoing.',
 'followup_question': 'What are the challenges in achieving nuclear fusion?'}

#### B. Json Mode

In [12]:
from langchain_openai import ChatOpenAI
import json
model = ChatOpenAI(model="gpt-4o-mini", model_kwargs={ "response_format": { "type": "json_object" } })
ai_msg = model.invoke("Return a JSON object with the format [{'method': <answer>, 'examples': <exeamples>},] explain methods of nuclear energy generation ")
json.loads(ai_msg.content)

{'methods': [{'method': 'Nuclear Fission',
   'examples': ['Pressurized Water Reactors (PWR)',
    'Boiling Water Reactors (BWR)',
    'Gas-Cooled Reactors (GCR)',
    'Fast Neutron Reactors (FNR)']},
  {'method': 'Nuclear Fusion',
   'examples': ['Tokamak Reactors',
    'Stellarators',
    'Inertial Confinement Fusion (ICF)',
    'Magnetized Target Fusion (MTF)']},
  {'method': 'Radioisotope Thermoelectric Generators (RTGs)',
   'examples': ['Power sources for space probes like Voyager and Curiosity',
    'Radioisotope heating units in remote applications',
    'Emergency power supplies in remote locations']},
  {'method': 'Nuclear Radiation Heating',
   'examples': ['Use of waste heat from reactor cores for district heating',
    'Heat exchange systems in molten salt reactors',
    'Research into nuclear heating for industrial processes']}]}

#### C. Structured Outputs and formatter

In [2]:
from pydantic import BaseModel, Field

class QueryFormatter(BaseModel):
    simple_queries: list[str] = Field(description="what simple questions could the user potentially want to ask?")
    advanced_queries: list[str] = Field(description="what advanced questions could the user potentially want to ask?")

class ResponseFormatter(BaseModel):
    feedback: str = Field(description="response to the query")
    further_queries: QueryFormatter = Field(default_factory=QueryFormatter)

In [30]:
# Bind the schema to the model
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
model_with_structure = model.with_structured_output(ResponseFormatter)
# Invoke the model
structured_output = model_with_structure.invoke("How is power generated using nuclear energy")


In [31]:
print(f"Response: {structured_output.feedback}")

Response: Nuclear energy is generated through a process called nuclear fission, where the nucleus of an atom, typically uranium-235 or plutonium-239, is split into smaller parts when it absorbs a neutron. This splitting releases a significant amount of energy in the form of heat. Here’s a step-by-step breakdown of how power is generated using nuclear energy:

1. **Nuclear Reactor**: The process takes place in a nuclear reactor, which contains fuel rods made of uranium or plutonium.

2. **Fission Reaction**: When a neutron collides with the nucleus of a fissile atom, it causes the nucleus to split, releasing energy, more neutrons, and fission products.

3. **Chain Reaction**: The released neutrons can then collide with other fissile nuclei, causing them to split as well, leading to a chain reaction. This reaction is carefully controlled in a reactor to maintain a steady output of energy.

4. **Heat Generation**: The energy released from fission heats up the reactor coolant, usually wate

In [32]:
print(f"Simple Queries: \n {structured_output.further_queries.simple_queries}")

Simple Queries: 
 ['What is nuclear fission?', 'What materials are used in nuclear reactors?', 'How does a nuclear reactor work?', 'What are the safety measures in nuclear power plants?', 'What are the advantages of nuclear energy?']


In [34]:
print(f"Advanced Queries: \n {structured_output.further_queries.advanced_queries}")

Advanced Queries: 
 ['What are the differences between nuclear fission and fusion?', 'How is waste from nuclear power plants managed?', 'What are the environmental impacts of nuclear energy?', 'How do different types of nuclear reactors compare in efficiency?', 'What advancements are being made in nuclear technology?']


In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
model_with_structure = model.with_structured_output(ResponseFormatter)

a_structured_output = await model_with_structure.ainvoke("How is power generated using nuclear energy")

In [10]:
print(a_structured_output.feedback)

Power generation using nuclear energy primarily involves the process of nuclear fission. In a nuclear reactor, heavy atomic nuclei, such as uranium-235 or plutonium-239, are split into smaller nuclei when they absorb a neutron. This fission process releases a significant amount of energy in the form of heat. Here’s a step-by-step breakdown of how this energy is converted into electricity:

1. **Nuclear Fission**: In the reactor core, fuel rods containing uranium or plutonium are bombarded with neutrons, causing the nuclei to split and release energy.

2. **Heat Generation**: The energy released from fission heats up the reactor coolant, usually water, which circulates through the reactor core.

3. **Steam Production**: The heated coolant is then used to produce steam in a secondary loop. In some designs, the coolant itself is turned into steam.

4. **Turbine Activation**: The steam generated is directed to a turbine, causing it to spin. This mechanical energy is crucial for the next st

In [11]:
print(f"Simple Queries: \n {a_structured_output.further_queries.simple_queries}")

Simple Queries: 
 ['What is nuclear fission?', 'How does a nuclear reactor work?', 'What are the advantages of nuclear energy?', 'What are the risks of nuclear power?', 'How is nuclear waste managed?']


In [12]:
print(f"Advanced Queries: \n {a_structured_output.further_queries.advanced_queries}")

Advanced Queries: 
 ['What are the differences between nuclear fission and fusion?', 'How do different types of nuclear reactors compare in efficiency?', 'What safety measures are in place in nuclear power plants?', 'How does the nuclear fuel cycle work?', 'What are the environmental impacts of nuclear energy?']
